In [1]:
import pandas as pd
import numpy as np
import random
import scipy.stats as ss

import datetime
import time
from datetime import datetime
from datetime import timedelta

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go # Import the graphical object

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#from IPython.core.pylabtools import figsize

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 40em; }</style>"))

pd.set_option('display.max_columns', 273)
pd.set_option('display.max_rows', 273)

In [2]:
countries = ['Brazil','Belgium','Argentina','France','England','Spain','Netherlands','Portugal',
'Denmark','Germany','Croatia','Mexico','Uruguay','Switzerland','United_States','Senegal',
'Wales','Iran','Serbia','Morocco','Japan','Poland','Korea','Tunisia','Costa_Rica','Australia',
'Canada','Cameroon','Ecuador','Qatar','Saudi_Arabia','Ghana']

countries_interest = ['Brazil','Argentina','England','Netherlands','Croatia','Uruguay','Wales','Ecuador','Qatar']

elos = [2169,2025,2143,2005,1920,2045,2040,2004,1971,1963,1927,1809,
1936,1902,1798,1687,1790,1797,1892,1753,1787,1814,1786,1707,1743,
1719,1776,1609,1833,1680,1635,1567]

coins_per_win = [1,1,1,1,2,2,2,2,4,4,4,4,8,8,8,8,16,16,16,16,32,32,32,32,64,64,64,64,128,128,128,128]

price = [75,26,60,56,45,50,36,30,23,39,17,12,17,12,11,12,11,8,13,9,9,10,9,8,8,8,9,9,9,8,8,9]

groups = [
['Qatar', 'Ecuador', 'Senegal', 'Netherlands'],
['England', 'Iran', 'United_States', 'Wales'],
['Argentina', 'Saudi_Arabia', 'Mexico', 'Poland'],
['France', 'Australia', 'Denmark', 'Tunisia'],
['Spain', 'Costa_Rica', 'Germany', 'Japan'],
['Belgium', 'Canada', 'Morocco', 'Croatia'],
['Brazil', 'Serbia', 'Switzerland', 'Cameroon'],
['Portugal', 'Ghana', 'Uruguay', 'Korea']]

letters = ['A','B','C','D','E','F','G','H']

group_winners = ['A1','A2','B1','B2','C1','C2','D1','D2','E1','E2','F1','F2','G1','G2','H1','H2']

tier_name_list = ['GROUP','R16','QF','SF_W','SF_L','FOURTH','THIRD','SECOND','WINNER']

# dictionary of Elo scores
# count of coins retunred by a team per win

dic_elos = dict(zip(countries, elos))
dic_coin_win = dict(zip(countries, coins_per_win))
dic_groups = dict(zip(letters, groups))
dic_price = dict(zip(countries, price))

In [3]:
df = pd.read_csv('team_progress.csv')
df.head()

,Unnamed: 0,result,country,tier,sim_no
0,0,A1,Netherlands,GROUP,0
1,1,A2,Senegal,GROUP,0
2,2,B1,England,GROUP,0
3,3,B2,Wales,GROUP,0
4,4,C1,Argentina,GROUP,0


# counts data Sankey

In [8]:
country = 'Ecuador'
group = 'A'
values = []

# prob exiting the groups
# prob of C1 or C2

df_group_1 = df[(df.country == country)&(df.result == group+'1')]
df_group_2 = df[(df.country == country)&(df.result == group+'2')]
values.append(df.sim_no.nunique() - (df_group_1.sim_no.nunique() + df_group_2.sim_no.nunique()))
values.append(df_group_1.sim_no.nunique())
values.append(df_group_2.sim_no.nunique())

# when country is C1, how likely are they to win
# of all the times a country is C1, how many times did they reach R16

df_R16_w_1 = df[(df.country == country)&(df.tier == 'R16')&(df.sim_no.isin(df_group_1.sim_no.unique()))]

values.append(df_R16_w_1.sim_no.nunique())
values.append(df_group_1.sim_no.nunique() - df_R16_w_1.sim_no.nunique())

# when country is C2, how likely are they to win
# of all the times a country is C2, how many times did they reach R16

df_R16_w_2 = df[(df.country == country)&(df.tier == 'R16')&(df.sim_no.isin(df_group_2.sim_no.unique()))]

values.append(df_R16_w_2.sim_no.nunique())
values.append(df_group_2.sim_no.nunique() - df_R16_w_2.sim_no.nunique())

# if they reach QF2, how likely are they to win

df_QF_w_1 = df[(df.tier == 'QF')&(df.country == country)&(df.sim_no.isin(df_R16_w_1.sim_no.unique()))]

values.append(df_QF_w_1.sim_no.nunique())
values.append(df_R16_w_1.sim_no.nunique() - df_QF_w_1.sim_no.nunique())

# if they reach QF4, how likely are they to win
# they are now in the semis

df_QF_w_2 = df[(df.tier == 'QF')&(df.country == country)&(df.sim_no.isin(df_R16_w_2.sim_no.unique()))]

values.append(df_QF_w_2.sim_no.nunique())
values.append(df_R16_w_2.sim_no.nunique() - df_QF_w_2.sim_no.nunique())

# if they reach SF1, how likely are they to win

df_SF_w_1 = df[(df.tier == 'SF_W')&(df.country == country)&(df.sim_no.isin(df_QF_w_1.sim_no.unique()))]
values.append(df_SF_w_1.sim_no.nunique())
values.append(df_QF_w_1.sim_no.nunique() - df_SF_w_1.sim_no.nunique())

# if they reach SF2, how likely are they to win

df_SF_w_2 = df[(df.tier == 'SF_W')&(df.country == country)&(df.sim_no.isin(df_QF_w_2.sim_no.unique()))]
values.append(df_SF_w_2.sim_no.nunique())
values.append(df_QF_w_2.sim_no.nunique() - df_SF_w_2.sim_no.nunique())

values.append(df[(df.country == country) & (df.tier == 'WINNER')].sim_no.nunique())
values.append(df[(df.country == country) & (df.tier == 'SECOND')].sim_no.nunique())
values.append(df[(df.country == country) & (df.tier == 'THIRD')].sim_no.nunique())
values.append(df[(df.country == country) & (df.tier == 'FOURTH')].sim_no.nunique())

node_label = ["GROUPS", "KO", "C1", "C2", "QF2", "QF4", "SF1", "SF2", "FINAL", "RUNNER_UP", "FIRST", "SECOND", "THIRD", "FOURTH"]
node_dict = {y:x for x, y in enumerate(node_label)}

source = ['GROUPS','GROUPS','GROUPS','C1', 'C1', 'C2', 'C2', 'QF2', 'QF2','QF4', 'QF4','SF1',  'SF1',      'SF2',  'SF2',       'FINAL', 'FINAL',  'RUNNER_UP','RUNNER_UP']
target = ['KO',    'C1',    'C2',    'QF2','KO', 'QF4','KO', 'SF1', 'KO', 'SF2', 'KO', 'FINAL','RUNNER_UP','FINAL','RUNNER_UP', 'FIRST', 'SECOND', 'THIRD',    'FOURTH'] 

source_node = [node_dict[x] for x in source]
target_node = [node_dict[x] for x in target]

fig = go.Figure( 
    data=[go.Sankey( # The plot we are interest
        # This part is for the node information
        node = dict( 
            label = node_label
        ),
        # This part is for the link information
        link = dict(
            source = source_node,
            target = target_node,
            value = values
        ))])

# With this save the plots 
plot(fig,
     image_filename='sankey_counts_'+country, 
     image='png', 
     image_width=1000, 
     image_height=600
)
fig.update_layout(title_text=country,
                  font_size=15)

# And shows the plot
fig.show()